In [ ]:
#UCB Project 1 Homelessness vs real estate prices
#Read and clean up Zillow Economics Data (1996-2017) 
#CITY DATA

In [114]:
#Import dependencies
import csv
from pandas import *
from IPython.display import display

In [150]:
#Read and assess the file
data_chunks = read_csv("City_time_series.csv", iterator=True, chunksize=500000)
city_prices_df= concat(data_chunks, ignore_index=True)
display(city_prices_df.head())

city_geo_df= read_csv("cities_crosswalk.csv")
city_geo_df= region_df.rename(columns = {'Unique_City_ID' : 'RegionName'})
display(city_geo_df.head())

,Date,RegionName,HomesSoldAsForeclosuresRatio_AllHomes,InventorySeasonallyAdjusted_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,...,ZHVI_BottomTier,ZHVI_CondoCoop,ZHVI_MiddleTier,ZHVI_SingleFamilyResidence,ZHVI_TopTier,ZRI_AllHomes,ZRI_AllHomesPlusMultifamily,ZriPerSqft_AllHomes,Zri_MultiFamilyResidenceRental,Zri_SingleFamilyResidenceRental
0,1996-04-30,abbottstownadamspa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,108700.0,NaN,NaN,NaN,NaN,NaN
1,1996-04-30,aberdeenharfordmd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,81300.0,137900.0,109600.0,108600.0,147900.0,NaN,NaN,NaN,NaN,NaN
2,1996-04-30,aberdeenmonroems,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,44700.0,44700.0,74500.0,NaN,NaN,NaN,NaN,NaN
3,1996-04-30,aberdeenmoorenc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,87400.0,131000.0,NaN,NaN,NaN,NaN,NaN
4,1996-04-30,abilenetaylortx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,RegionName,City,County,State
0,oak_grovechristianky,Oak Grove,Christian,KY
1,jarvisburgcurritucknc,Jarvisburg,Currituck,NC
2,mcminnvilleyamhillor,McMinnville,Yamhill,OR
3,union_townshiperiepa,Union Township,Erie,PA
4,oshkoshwinnebagowi,Oshkosh,Winnebago,WI


In [151]:
#Output anatomy of files
print ('Total # of rows in df: %s' % (len(city_prices_df)))
print ('Total # of columns in df: %s' % len(city_prices_df.columns))
#df.info()
#print(list(city_prices_df))

print ('Total # of rows in region_df: %s' % (len(city_geo_df)))
print ('Total # of columns in region_df: %s' % len(city_geo_df.columns))
#df.info()
#print(list(city_geo_df))

Total # of rows in df: 4172721
Total # of columns in df: 85
Total # of rows in region_df: 25341
Total # of columns in region_df: 4


In [154]:
#Create a new df with the columns of interest
# HVI - a smoothed seasonally adjusted measure of the median estimated home value across a given region and housing type
Zillow_df = city_prices_df[["Date","RegionName","MedianSoldPricePerSqft_AllHomes","MedianSoldPricePerSqft_SingleFamilyResidence",
               "MedianSoldPrice_AllHomes","ZHVIPerSqft_AllHomes","ZHVI_AllHomes", "ZHVI_SingleFamilyResidence"]]  
                            
#display(Zillow_df)

#Merge the two dataframes
merged_df = pd.merge(city_geo_df, Zillow_df, on="RegionName")
display(merged_df)

#Checkout merged_df output
print ('Total # of rows in df: %s' % (len(merged_df)))
print ('Total # of columns in df: %s' % len(merged_df.columns))
#merged_df.info()

,RegionName,City,County,State,Date,MedianSoldPricePerSqft_AllHomes,MedianSoldPricePerSqft_SingleFamilyResidence,MedianSoldPrice_AllHomes,ZHVIPerSqft_AllHomes,ZHVI_AllHomes,ZHVI_SingleFamilyResidence
0,oak_grovechristianky,Oak Grove,Christian,KY,1996-04-30,NaN,NaN,NaN,62.0,70000.0,70000.0
1,oak_grovechristianky,Oak Grove,Christian,KY,1996-05-31,NaN,NaN,NaN,62.0,69900.0,69900.0
2,oak_grovechristianky,Oak Grove,Christian,KY,1996-06-30,NaN,NaN,NaN,62.0,69700.0,69700.0
3,oak_grovechristianky,Oak Grove,Christian,KY,1996-07-31,NaN,NaN,NaN,62.0,70000.0,70000.0
4,oak_grovechristianky,Oak Grove,Christian,KY,1996-08-31,NaN,NaN,NaN,62.0,70100.0,70100.0
5,oak_grovechristianky,Oak Grove,Christian,KY,1996-09-30,NaN,NaN,NaN,62.0,70000.0,70000.0
6,oak_grovechristianky,Oak Grove,Christian,KY,1996-10-31,NaN,NaN,NaN,61.0,69900.0,69900.0
7,oak_grovechristianky,Oak Grove,Christian,KY,1996-11-30,NaN,NaN,NaN,61.0,69700.0,69700.0
8,oak_grovechristianky,Oak Grove,Christian,KY,1996-12-31,NaN,NaN,NaN,61.0,69100.0,69100.0
9,oak_grovechristianky,Oak Grove,Christian,KY,1997-01-31,NaN,NaN,NaN,61.0,68400.0,68400.0


Total # of rows in df: 4172721
Total # of columns in df: 11


In [153]:
#Remove data prior to 2007
#"Date" is an object
#merged_df.drop(pd.to_datetime('1996-04-30'))

#Set date as index
#merged_df=merged_df.set_index('Date')
#grouped= merged_df.groupby(["Date"])

# mod_merged=grouped.loc[['Date'] < 2007, ["RegionName":"ZHVI_SingleFamilyResidence"]]
# display(grouped.count())

In [158]:
#Select cities, based on cross-examination of data available from HUD dataset
Cities = ["New York", "Philadelphia", "Boston", "Washington", "Chicago", "Minneapolis",
        "Denver", "Salt Lake City", "Seattle", "Los Angeles", "San Francisco", "Miami", 
        "Charlotte", "Atlanta", "Detroit", "Anchorage", "Honolulu", "Indianapolis"]

#Create a new dataframe only with data for the selected cities -WORK IN PROGRESS
for row in merged_df:
    if city in Cities:
        df.loc['City', 'City':'ZHVI_SingleFamilyResidence']


#Clean dataframe - remove data before 2007 and remove columns mostly empty
# df.dropna(axis=1, how='any', inplace=True)
# display(df)
# print ('Total # of rows: %s' % (len(Zillow_df)))
# print ('Total # of columns: %s' % len(Zillow_df.columns))

#Export as csv file

NameError: name 'city' is not defined